In [1]:
from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora import Dictionary
from gensim.models.ldamodel import LdaModel
import gensim
import pickle
import pandas as pd
import numpy as np

C:\Anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


### Read in each topic model.  Get the topic/word matrix.  Analyze the probabilities by word

The topic models were created in the "LDA with gensim" notebook

In [2]:
dictionary = Dictionary.load('movies_dictionary.gensim')


In [3]:
fname = 'movies_topics_5.gensim'
ldatmp = LdaModel.load(fname) #, mmap='r')
topic_matrix = ldatmp.get_topics()


In [5]:
#topic_matrix.shape
# (5, 310703)
# Sum up probabilities for each word.  I don't know if this is strictly correct, but it seems to be useful
words = topic_matrix.sum(axis=0)

In [12]:
print(words.argmax())
print(words[words.argmax()])
print(dictionary.id2token[words.argmax()])


200
0.09066486
the


The word "the" shouldn't appear in the dictionary.  This told me there was something wrong with my stop word processing.  I should have used lower() before processing

In [13]:
indices = np.argsort(words, axis=0)

In [21]:
for i in indices[-20:]:
    print (dictionary.id2token[i], words[i])

would 0.013930077
even 0.013995692
first 0.0142959645
much 0.014359055
make 0.014730598
book 0.016440477
scene 0.016828872
also 0.017105663
great 0.01711501
this 0.017664328
well 0.018167106
good 0.020014334
like 0.022106498
time 0.02227082
story 0.023099009
character 0.023885917
one 0.036454484
movie 0.05619169
film 0.080125116
the 0.09066486


In [28]:
from collections import defaultdict

In [31]:
# These will only exist if you executed the LDA notebook.  I'm not currently checking these in since they're exploratory
TOPIC_SIZES = [5, 10, 15, 20, 25, 30, 40, 50, 60]
results_cv = []
word_dict = defaultdict(int)

for topic_size in TOPIC_SIZES:
    print ('topic size: ', topic_size)
    fname = 'movies_topics_' + str(topic_size) + '.gensim'
    ldatmp = LdaModel.load(fname) #, mmap='r')
    topic_matrix = ldatmp.get_topics()
    words = topic_matrix.sum(axis=0) # sum up each word's probability
    indices = np.argsort(words, axis=0)
    for i in indices[-20:]:
        print (dictionary.id2token[i], words[i])
        word_dict[dictionary.id2token[i]] = word_dict[dictionary.id2token[i]] + 1
    

topic size:  5
would 0.013930077
even 0.013995692
first 0.0142959645
much 0.014359055
make 0.014730598
book 0.016440477
scene 0.016828872
also 0.017105663
great 0.01711501
this 0.017664328
well 0.018167106
good 0.020014334
like 0.022106498
time 0.02227082
story 0.023099009
character 0.023885917
one 0.036454484
movie 0.05619169
film 0.080125116
the 0.09066486
topic size:  10
best 0.027888313
make 0.028402815
get 0.028670745
scene 0.030265296
see 0.03033463
bond 0.03127066
well 0.03158938
great 0.033602793
also 0.03361028
love 0.033639748
this 0.03673403
good 0.0401398
character 0.041948467
time 0.042088628
story 0.04238884
like 0.044474747
one 0.07356834
movie 0.123443045
film 0.13525663
the 0.1612034
topic size:  15
scene 0.037786927
action 0.038541652
novel 0.038737312
good 0.0387716
tom 0.039147764
this 0.03944949
also 0.04031747
read 0.04056485
world 0.04118835
war 0.043827083
time 0.046212796
character 0.047678333
bond 0.047996424
like 0.04805067
story 0.04855518
one 0.07973199
mov

## This shows how many time each word was in the top 20 for each model

In [32]:
word_dict

defaultdict(int,
            {'would': 1,
             'even': 1,
             'first': 2,
             'much': 1,
             'make': 2,
             'book': 8,
             'scene': 5,
             'also': 6,
             'great': 2,
             'this': 7,
             'well': 2,
             'good': 4,
             'like': 9,
             'time': 9,
             'story': 9,
             'character': 9,
             'one': 9,
             'movie': 9,
             'film': 9,
             'the': 9,
             'best': 3,
             'get': 1,
             'see': 1,
             'bond': 6,
             'love': 3,
             'action': 3,
             'novel': 1,
             'tom': 5,
             'read': 2,
             'world': 4,
             'war': 5,
             'hitchcock': 2,
             'spielberg': 5,
             'cruise': 4,
             'vampire': 5,
             'child': 1,
             'moore': 1,
             'game': 2,
             'edward': 3,
             'bella

## This is separate analysis to count the word frequencies in the cleaned text

In [33]:
reviews = pd.read_csv("..\\Data_Processing\\cleaned.csv",encoding="utf-8")
texts = reviews["cleaned"].apply(lambda s: s.split(' '))

In [34]:
corpusdict = defaultdict(int)
for text in texts:
    textdict = {}
    for word in text:
        if word not in textdict:
            textdict[word] = 1
    for word in textdict:
        corpusdict[word] = corpusdict[word] + 1


In [38]:
totaltexts = len(texts) * 1.0

In [39]:
totaltexts

263535.0

In [59]:
for k,v in corpusdict.items():
    if v/totaltexts > .5: # and v/totaltexts < .6:
        print (k, v/totaltexts)

one 0.5819682395127782
movie 0.708812871155634
film 0.6298935625249018
the 0.7475629423036788
